In [1]:
import utils
import math
import re
import itertools
from typing import NamedTuple
from collections import defaultdict, deque
import itertools

import matplotlib.pyplot as plt

## Day 16: Ticket Translation

[#](https://adventofcode.com/2020/day/16).  

In [2]:
test16 = """class: 1-3 or 5-7
row: 6-11 or 33-44
seat: 13-40 or 45-50

your ticket:
7,1,14

nearby tickets:
7,3,47
40,4,50
55,2,20
38,6,12"""

inp16 = utils.get_input(16, splitlines=False)

Parsing the input:

In [3]:
def parse_16(inp=test16, verbose=False):
    notes = dict()
    nearby_tickets = []

    _notes, _ticket, _nearby_tickets = inp.split("\n\n")

    for note in _notes.splitlines():
        name, vals = note.split(": ")
        notes[name] = [int(n) for n in re.findall(r"\d+", vals)]
    if verbose: print(notes)

    ticket = [int(i) for i in _ticket.splitlines()[1].split(",")]
    if verbose: print(ticket)

    for line in _nearby_tickets.splitlines()[1:]:
        nearby_tickets.append([int(i) for i in line.split(",")])
    if verbose: print(nearby_tickets)
    
    return notes, ticket, nearby_tickets

notes, ticket, nearby_tickets = parse_16(verbose=True)

{'class': [1, 3, 5, 7], 'row': [6, 11, 33, 44], 'seat': [13, 40, 45, 50]}
[7, 1, 14]
[[7, 3, 47], [40, 4, 50], [55, 2, 20], [38, 6, 12]]


In [12]:
def solve_16(inp=test16):
    notes, ticket, nearby_tickets = parse_16(inp)
    errors = []
    
    def check_field(num):
        valid = []
        for key, check_ranges in notes.items():
            a, b, x, y = check_ranges
            valid.append(a <= num <= b or x <= num <= y)

        if any(valid):
            return None
        else:
            return num
    
    for t in nearby_tickets:
        for n in t:
            if err := check_field(n):
                errors.append(err)

    return sum(errors)

assert(solve_16() == 71)
solve_16(inp16)

23122

## Part 2

Now we're discarding invalid tickets, and using the remaining tickets to figure out which field is what.


In [15]:
test16b = """class: 0-1 or 4-19
row: 0-5 or 8-19
seat: 0-13 or 16-19

your ticket:
11,12,13

nearby tickets:
3,9,18
15,1,5
5,14,9"""

In [22]:
notes, ticket, nearby_tickets = parse_16(test16b, True)

{'class': [0, 1, 4, 19], 'row': [0, 5, 8, 19], 'seat': [0, 13, 16, 19]}
[11, 12, 13]
[[3, 9, 18], [15, 1, 5], [5, 14, 9]]


In [23]:
def valid_ticket(ticket):
    
    def check_field(num):
        valid = []
        for key, check_ranges in notes.items():
            a, b, x, y = check_ranges
            valid.append(a <= num <= b or x <= num <= y)

        if any(valid):
            return None
        else:
            return num
        
    return not any([check_field(n) for n in ticket])

for t in nearby_tickets:
    print(t, valid_ticket(t))

[3, 9, 18] True
[15, 1, 5] True
[5, 14, 9] True


In [24]:
def a(num):
    valid = []
    for key, check_ranges in notes.items():
        a, b, x, y = check_ranges
        valid.append(a <= num <= b or x <= num <= y)
    
    return valid

In [25]:
for num in t:
    print(a(num))

[True, True, True]
[True, True, False]
[True, True, True]


In [48]:
import pandas as pd
import numpy as np

In [30]:
all_tickets = [ticket] + nearby_tickets

In [32]:
check = defaultdict(list)

for tick in all_tickets:
    for i, num in enumerate(tick):
        check[i].append(a(num))

In [51]:
df = pd.DataFrame(check[0])
df

,0,1,2
0,True,True,True
1,False,True,True
2,True,True,False
3,True,True,True


In [60]:
df

,0,1,2
0,True,True,True
1,False,True,True
2,True,True,False
3,True,True,True


In [55]:
for i, t in enumerate(df.sum(axis=0)):
    print(i, t)

0 3
1 4
2 3
